<a href="https://colab.research.google.com/github/mehmeh-ctrl/python-codewars/blob/master/headlines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
maplist=[]
with open('https://gist.githubusercontent.com/mehmeh-ctrl/0072e2b4558131b2240e2304775118a1/raw/1379eacdfc3cba8023299520375314a7065d5ad1/headlines.txt', 'r') as headlines:
    for line in headlines:
        maplist.append(line.rstrip('\n'))

In [0]:
for headline in maplist:
    print(headline)

In [0]:
cols=["headline", "city", "country"]

In [0]:
import pandas as pd
headlines_df=pd.DataFrame(columns=cols)

In [0]:
!pip install geonamescache

In [0]:
import geonamescache

from geonamescache import GeonamesCache
gc = GeonamesCache()

In [0]:
cities = gc.get_cities()

In [0]:
countries = gc.get_countries()

In [0]:
cities_names=[]
cities_id=list(cities.keys())
for city_id in cities_id:
    cities_names.append(cities[city_id]['name'])

In [0]:
countries_names=[]
countries_id=list(countries.keys())
for country_id in countries_id:
    countries_names.append(countries[country_id]['name'])

In [0]:
!pip install unidecode
from unidecode import unidecode
accented_names = [city['name'] for city in gc.get_cities().values()
                  if city['name'] != unidecode(city['name'])]

In [0]:
cities_names_unidecoded=[unidecode(city) for city in cities_names]

In [0]:
countries_unidecoded=[unidecode(country) for country in countries_names]

In [0]:
countries_unidecoded_new=[]
for country in countries_unidecoded:
    if ',' in country:
        country=country.split(',')
        country_str = country[1] + country[0]
        new_country = ''.join(country_str)
        countries_unidecoded_new.append(new_country)
    else:
        countries_unidecoded_new.append(country)

In [0]:
import re

dict_city=dict()
dict_country=dict()
for city_name in cities_names_unidecoded:
    regexp_city_name = re.compile(r'\b'+city_name+r'\b')
    dict_city[city_name]= regexp_city_name
for country in countries_unidecoded_new:
    regexp_country_name=re.compile(r'\b'+country+r'\b')
    dict_country[country] =  regexp_country_name

In [0]:
import numpy as np
import pandas as pd

for city_name in cities_names_unidecoded:
    for headline in maplist:
        if dict_city[city_name].search(headline):
            for country in countries_unidecoded_new:
                if dict_country[country].search(headline):
                    a_row = pd.Series([headline, city_name, country])
                    row_df = pd.DataFrame([a_row])
                    headlines_df = pd.concat([row_df, headlines_df], ignore_index=True)
                else:
                    a_row = pd.Series([headline, city_name, np.nan])
                    row_df = pd.DataFrame([a_row])
                    headlines_df = pd.concat([row_df, headlines_df], ignore_index=True)
        else:
            for country in countries_unidecoded_new:
                if dict_country[country].search(headline):
                    a_row = pd.Series([headline, np.nan, country])
                    row_df = pd.DataFrame([a_row])
                    headlines_df = pd.concat([row_df, headlines_df], ignore_index=True)
                else:
                    a_row = pd.Series([headline, np.nan, np.nan])
                    row_df = pd.DataFrame([a_row])
                    headlines_df = pd.concat([row_df, headlines_df], ignore_index=True)
        
print(headlines_df)